In [1]:
import math
import numpy as np
import torch
import sys
import torch.nn as nn
import torch.nn.functional as F
from timeit import default_timer
import sys
import os
sys.path.append("../../")
from utility.adam import Adam
from utility.losses import LpLoss
from utility.normalizer import UnitGaussianNormalizer
from pcno.geo_utility import compute_edge_gradient_weights

In [2]:

equal_weights = False
data_path = "../../data/adv_diff_bvp/"
data = np.load(data_path+"pcno_data.npz")
nnodes, node_mask, nodes = data["nnodes"], data["node_mask"], data["nodes"]
node_weights = data["node_equal_weights"] if equal_weights else data["node_weights"]
directed_edges, edge_gradient_weights = data["directed_edges"], data["edge_gradient_weights"]
features = data["features"]
node_measures = data["node_measures"]
node_measures_raw = data["node_measures_raw"]
indices = np.isfinite(node_measures_raw)
node_rhos = np.copy(node_weights)
node_rhos[indices] = node_rhos[indices]/node_measures[indices]


nnodes = torch.from_numpy(nnodes)
node_mask = torch.from_numpy(node_mask)
nodes = torch.from_numpy(nodes.astype(np.float32))
node_weights = torch.from_numpy(node_weights.astype(np.float32))
node_rhos = torch.from_numpy(node_rhos.astype(np.float32))
features = torch.from_numpy(features.astype(np.float32))
directed_edges = torch.from_numpy(directed_edges.astype(np.int64))
edge_gradient_weights = torch.from_numpy(edge_gradient_weights.astype(np.float32))

nodes_input = nodes.clone()
N = nnodes.shape[0]
x_test = torch.cat((features[:,:,[0,2,3]],nodes_input, node_rhos), -1)
aux_test = (node_mask,  nodes,  node_weights,  directed_edges,  edge_gradient_weights)
y_test = features[:, :, [1]]
print('x_test.shape: ',x_test.shape,' y_test.shape: ',y_test.shape , 'node_mask.shape: ',node_mask.shape)
test_tuple = (x_test,y_test,node_mask,nodes,node_weights,directed_edges,edge_gradient_weights)

x_test.shape:  torch.Size([7500, 7501, 5])  y_test.shape:  torch.Size([7500, 7501, 1]) node_mask.shape:  torch.Size([7500, 7501, 1])


In [14]:
import matplotlib.pyplot as plt
from contextlib import redirect_stdout
from pcno.pcno import compute_Fourier_modes, PCNO, PCNO_train

def test_model(model,n_test,test_tuple):

    x_test,y_test,node_mask,nodes,node_weights,directed_edges,edge_gradient_weights = test_tuple
    rel_l2_uniform = []
    index_uniform = []
    rel_l2_exponential = []
    index_exponential = []
    rel_l2_linear = []
    index_linear = []
    myloss = LpLoss(d=1, p=2, size_average=False)
    with torch.no_grad():
        for i in range(N-n_test,N):
            
            x, y = x_test[i:i+1].to(device), y_test[i:i+1].to(device)
            aux_batch = (
            node_mask[i:i+1].to(device), nodes[i:i+1].to(device),
            node_weights[i:i+1].to(device), directed_edges[i:i+1].to(device),
            edge_gradient_weights[i:i+1].to(device)
            )

            out = model(x, aux_batch) #.reshape(batch_size_,  -1)

            batch_size_ = x.shape[0]
            out=out*node_mask[i:i+1].to(device) #mask the padded value with 0,(1 for node, 0 for padding)
            test_rel_l2 = myloss(out.view(batch_size_,-1), y.view(batch_size_,-1)).item()
            # test_l2 = myloss.abs(out.view(batch_size_,-1), y.view(batch_size_,-1)).item()
            if i%3==0:
                rel_l2_uniform.append(test_rel_l2)
                index_uniform.append(i)
            elif i%3 ==1:
                rel_l2_exponential.append(test_rel_l2)
                index_exponential.append(i)
            else:
                rel_l2_linear.append(test_rel_l2)
                index_linear.append(i)            
            # print(f'test index: {i}, test_l2: {test_l2}, test_rel_l2: {test_rel_l2}, test_type: {type_dict[i%3]}')
    return   rel_l2_uniform,  rel_l2_exponential, rel_l2_linear, index_uniform, index_exponential, index_linear

def sorted_result( rel_l2_uniform,  rel_l2_exponential, rel_l2_linear, index_uniform, index_exponential, index_linear):


    uniform_sorted = sorted(enumerate(rel_l2_uniform), key=lambda x: x[1], reverse=True)
    exponential_sorted = sorted(enumerate(rel_l2_exponential), key=lambda x: x[1], reverse=True)
    linear_sorted = sorted(enumerate(rel_l2_linear), key=lambda x: x[1], reverse=True)

    average_loss_list = [sum(rel_l2_uniform)/len(rel_l2_uniform),sum(rel_l2_exponential)/len(rel_l2_exponential),sum(rel_l2_linear)/len(rel_l2_linear)]
    average_loss_list.append((sum(average_loss_list)/len(average_loss_list)))
    print('average rel_l2 of uniform:  ',round(average_loss_list[0],5) ,flush = True)
    print('average rel_l2 of exponential:  ',round(average_loss_list[1],5))
    print('average rel_l2 of linear:  ',round(average_loss_list[2],5))
    print('average rel_l2 of mixed:  ',round(average_loss_list[3],5))

    print()


    n = 1
    index_uniform_3 = [index_uniform[uniform_sorted[0][0]],index_uniform[uniform_sorted[len(uniform_sorted)//2][0]],index_uniform[uniform_sorted[-1][0]]]
    index_exponential_3 = [index_exponential[exponential_sorted[0][0]],index_exponential[exponential_sorted[len(exponential_sorted)//2][0]],index_exponential[exponential_sorted[-1][0]]]
    index_linear_3 = [index_linear[linear_sorted[0][0]],index_linear[linear_sorted[len(linear_sorted)//2][0]],index_linear[linear_sorted[-1][0]]]
    for j in range(n):
        print(f'{j+1}th worst rel_l2 of uniform:  ',round(uniform_sorted[j][1],5), ' index: ',index_uniform[uniform_sorted[j][0]])
        print(f'{j+1}th worst rel_l2 of exponential:  ',round(exponential_sorted[j][1],5), ' index: ',index_exponential[exponential_sorted[j][0]])
        print(f'{j+1}th worst rel_l2 of linear:  ',round(linear_sorted[j][1],5), ' index: ',index_linear[linear_sorted[j][0]])
        print()
    print('medium rel_l2 of uniform: ',round(uniform_sorted[len(uniform_sorted)//2][1],5), ' index: ',index_uniform_3[1])
    print('medium rel_l2 of exponential: ',round(exponential_sorted[len(exponential_sorted)//2][1],5), ' index: ',index_exponential_3[1])
    print('medium rel_l2 of linear: ',round(linear_sorted[len(linear_sorted)//2][1],5), ' index: ',index_linear_3[1])
    print()
    for j in range(n):    
        print(f'{j+1}th best rel_l2 of uniform:  ',round(uniform_sorted[-j-1][1],5), ' index: ',index_uniform[uniform_sorted[-j-1][0]])
        print(f'{j+1}th best rel_l2 of exponential:  ',round(exponential_sorted[-j-1][1],5), ' index: ',index_exponential[exponential_sorted[-j-1][0]])
        print(f'{j+1}th best rel_l2 of linear:  ',round(linear_sorted[-j-1][1],5), ' index: ',index_linear[linear_sorted[-j-1][0]],flush = True)

 
        
    return average_loss_list,index_uniform_3,index_exponential_3,index_linear_3

def myplot(average_loss_list,index_uniform_3,index_exponential_3,index_linear_3,save_figure_path,
           model,test_tuple):

    x_test,y_test,node_mask,nodes,node_weights,directed_edges,edge_gradient_weights = test_tuple
    myloss = LpLoss(d=1, p=2, size_average=False)
    index_dict = {0:index_uniform_3,1:index_exponential_3,2:index_linear_3}
    type_dict = {0:'uniform', 1:'exponential', 2:'linear'}

    with torch.no_grad():
        for k in range(3):
            index_list = index_dict[k]
            test_type = type_dict[k]

            fig, axs = plt.subplots(3, 3, figsize=(12, 12))
            for col, title in enumerate(['truth', 'prediction', f'error, average = {round(average_loss_list[k],4)}']):
                axs[0, col].set_title(title)
            for j in range(len(index_list)):
                i = index_list[j]
                x, y = x_test[i:i+1].to(device), y_test[i:i+1].to(device)
                aux_batch = (
                node_mask[i:i+1].to(device), nodes[i:i+1].to(device),
                node_weights[i:i+1].to(device), directed_edges[i:i+1].to(device),
                edge_gradient_weights[i:i+1].to(device)
                )
                out = model(x, aux_batch) #.reshape(batch_size_,  -1)

                batch_size_ = x.shape[0]
                out = out*node_mask[i:i+1].to(device) #mask the padded value with 0,(1 for node, 0 for padding)
                test_rel_l2 = myloss(out.view(batch_size_,-1), y.view(batch_size_,-1)).item()



                axs[j, 0].plot(nodes[i], y_test[i], "o", markersize=1)
                # axs[j, 0].set_title('truth')
                axs[j, 1].plot(nodes[i], out.reshape(-1).detach().cpu(), "o", markersize=1)
                # axs[j, 1].set_title('prediction')

                error = out.reshape(-1).detach().cpu()-y_test[i].reshape(-1)
                axs[j, 2].plot(nodes[i], error, "o", markersize=1,label = f'loss={round(test_rel_l2,3)}')
                axs[j, 2].legend()
            plt.tight_layout(rect=[0, 0.03, 1, 0.95])
            if not os.path.exists(save_figure_path):
                os.makedirs(save_figure_path)
            fig.savefig(save_figure_path + f'test_{test_type}.png', format='png', bbox_inches='tight') 
            # fig.savefig(f'train_{model_train_type}_test_{test_type}.pdf', format='pdf', bbox_inches='tight') 
            plt.close(fig)

n_test = 600
device = 'cuda'
# model_train_type_dict = ['uniform', 'exponential','linear', 'mixed']
# model_n_train_dict =[500,1000,1500]
model_train_sp_L_dict = [False, 'together', 'independently']
model_train_type_dict = ['mixed']
model_n_train_dict =[1000]
# model_train_sp_L_dict = [False]
equal_weight = False

with open('notes/output.txt', 'w') as f:
    with redirect_stdout(f):
        for model_train_type in model_train_type_dict:
            for model_n_train in model_n_train_dict:
                for model_train_sp_L in model_train_sp_L_dict:

                    if equal_weight:
                        model_path = f'model/pcno_adv_{model_n_train}_equal_weight/{model_train_type}_{model_train_sp_L}.pth'
                    else:
                        model_path = f'model/pcno_adv_{model_n_train}/{model_train_type}_{model_train_sp_L}.pth'
                    k_max = 32
                    ndim = 1


                    modes = compute_Fourier_modes(ndim, [k_max], [15.0])
                    modes = torch.tensor(modes, dtype=torch.float).to(device)
                    model = PCNO(ndim, modes, nmeasures=1,
                                layers=[128,128,128,128,128],
                                fc_dim=128,
                                in_dim=x_test.shape[-1], out_dim=y_test.shape[-1],
                                train_sp_L = model_train_sp_L,
                                act='gelu').to(device)
                    checkpoint = torch.load(model_path, map_location=device)
                    model.load_state_dict(checkpoint)
                    print('\nload model state dict from ' + model_path, flush = True)
                    save_figure_path = f'figure/pcno_adv_{model_n_train}/{model_train_type}_{model_train_sp_L}/'

                    rel_l2_uniform,  rel_l2_exponential, rel_l2_linear, index_uniform, index_exponential, index_linear = test_model(model,n_test,test_tuple)
                    average_loss_list,index_uniform_3,index_exponential_3,index_linear_3 = sorted_result( rel_l2_uniform,  rel_l2_exponential, rel_l2_linear, index_uniform, index_exponential, index_linear)
                    myplot(average_loss_list,index_uniform_3,index_exponential_3,index_linear_3,save_figure_path,
                            model,test_tuple)




In [16]:
import re

def parse_file(file_path):
    '''
    data[datasize][loss_type][model_type]
    datasize:  500, 1000, 1500
    loss_type: average, worst
    model_type: exponential_False , ...
    '''
    with open(file_path, 'r') as file:
        lines = file.readlines()

    if "equal_weight" in lines[0]:
        data = {
            1000: 
            {
            'average': {},
            'worst': {}
            },  
            equal_weight: True
        }
    else:
        data = {
            500:
            {
            'average': {},
            'worst': {}
            },
            1000:
            {
            'average': {},
            'worst': {}
            },
            1500:
            {
            'average': {},
            'worst': {}
            },
             equal_weight: False
            
        }
    current_model = None
    for line in lines:
        line = line.strip()
        if line.startswith('load model'):
            # Extract the model name from the line.
            match = re.search(r'load model state dict from (\S+)', line)
            if match:
                current_model = match.group(1).split('/')[-1].replace('.pth', '')
                datasize = int(next(item for item in match.group(1).split('/')[1].split('_') if item.isdigit()))
                data[datasize]['average'][current_model] = {}
                data[datasize]['worst'][current_model] = {}

        elif current_model and 'average rel_l2 of ' in line:
            key = line.split()[-2].replace(':', '')
            value = line.split()[-1]
            data[datasize]['average'][current_model][key.strip()] = float(value.strip())

        elif current_model and '1th worst rel_l2 of ' in line:
            key = line.split()[-4].replace(':', '')
            value = line.split()[-3]
            data[datasize]['worst'][current_model][key.strip()] = {'1th_worst': float(value.strip().split()[0])}

    return data


def generate_markdown_table(data, datasize, table_type):
    def format_row(keys, row_data):
        return " | ".join([str(row_data.get(key, '-')) for key in keys])

    header = "|train_type | uniform | exponential | linear | mixed |\n"
    format_header = "| :---: | :---: | :---: | :---: | :---: |\n"

    rows = []
    train_types = sorted(data[datasize][table_type].keys())
    columns = ['uniform', 'exponential', 'linear', 'mixed']

    for train_type in train_types:
        row_data = data[datasize][table_type][train_type]
        if table_type == 'average':
            formatted_row = f"| {train_type} | {format_row(columns, row_data)} |"
        elif table_type == 'worst':
            formatted_row = f"| {train_type} | {format_row(columns, {col: val['1th_worst'] for col, val in row_data.items()})} |"
        rows.append(formatted_row)

    if table_type == 'average':
        title = f'# average loss of datasize {datasize}, equal_weight = {data[equal_weight]}\n'
    elif table_type == 'worst':
        title = f'# biggest loss of datasize {datasize}\n'

    return title + header + format_header + "\n".join(rows) + "\n\n"


# Parse the input file
parsed_data = parse_file('notes/output.txt')
print(parsed_data)

# Optionally write to files
with open('notes/loss1.md', 'w') as f:
    for datasize in [500,1000,1500]:
        avg_loss_md = generate_markdown_table(parsed_data, datasize, 'average')
        # Print or save to files
        print(avg_loss_md)
        f.write(avg_loss_md)

    for datasize in [500,1000,1500]:
        worst_loss_md = generate_markdown_table(parsed_data, datasize, 'worst')
        print(worst_loss_md)
        f.write(worst_loss_md)


    


{500: {'average': {}, 'worst': {}}, 1000: {'average': {'mixed_False_state_dict': {'uniform': 0.01727, 'exponential': 0.05875, 'linear': 0.02777, 'mixed': 0.0346}, 'mixed_together_state_dict': {'uniform': 0.01733, 'exponential': 0.06761, 'linear': 0.02898, 'mixed': 0.03798}, 'mixed_independently_state_dict': {'uniform': 0.01796, 'exponential': 0.06853, 'linear': 0.03107, 'mixed': 0.03918}}, 'worst': {'mixed_False_state_dict': {'uniform': {'1th_worst': 0.05978}, 'exponential': {'1th_worst': 0.14071}, 'linear': {'1th_worst': 0.0748}}, 'mixed_together_state_dict': {'uniform': {'1th_worst': 0.05874}, 'exponential': {'1th_worst': 0.15225}, 'linear': {'1th_worst': 0.07215}}, 'mixed_independently_state_dict': {'uniform': {'1th_worst': 0.06119}, 'exponential': {'1th_worst': 0.14775}, 'linear': {'1th_worst': 0.089}}}}, 1500: {'average': {}, 'worst': {}}}
# average loss of datasize 500
|train_type | uniform | exponential | linear | mixed |
| :---: | :---: | :---: | :---: | :---: |



# average lo